In [ ]:
### Import libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
### Load data

### Adjust the command according to the name and directory of your data

# file_path = '...your directory here.../data.txt' 

### It's important to check if the decimal place is defined by a "." or ",". 
### Adjust as needed.
df_all = pd.read_csv(file_path, sep='\t', header=0, decimal='.') 


### Adjust the entries according to the data or, if you prefer, modify the name of each column. 
### AMB = environment, GEN = genotype, BLO = block or repetition, PROD = yield.
df = df_all[['AMB', 'GEN', 'BLO', 'PROD']]


df = df.groupby(['AMB', 'GEN'])[['PROD']].mean().reset_index()

### Round the values ​​to two decimal places
df = df.round({'PROD': 2})

#df.to_csv('genotype_yield.txt', sep='\t', decimal='.', index=False)
df.head(10)

In [ ]:
### ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols

df_ANOVA = df_all[['AMB', 'GEN', 'BLO', 'PROD']]
df_ANOVA = df_ANOVA.groupby(['AMB', 'GEN', 'BLO'])[['PROD']].mean().reset_index()

r = len(df_ANOVA['BLO'].unique())
model = ols("PROD ~ C(BLO):C(AMB) + C(AMB) + C(GEN) + C(AMB):C(GEN)", data=df_ANOVA).fit()

### Use typ=2 for the average sum of squares
anova_table = sm.stats.anova_lm(model, typ=2)  
anova_table["QM"] = anova_table["sum_sq"] / anova_table["df"]

### # Replace "PROD" with your response variable
mean_prod = df_ANOVA["PROD"].mean()  

### model.mse_resid It directly provides the QMR (root mean square of the residuals).
qmr = model.mse_resid  

QMR_global = qmr
cv = (qmr**0.5 / mean_prod) * 100
print("\nMean Squares (MS) per term:")
for term, qm in anova_table["QM"].items():
    print(f"{term}: {qm:.4f}")

print(f"Coefficient of Variation (CV%): {cv:.2f}")
print(f"Overall mean: {mean_prod:.2f}")


In [ ]:
### Frequentist method for determining a priori information

import pandas as pd
import numpy as np
import statsmodels.api as sm
import time

### Assume that df contains the columns 'GEN', 'AMB', and 'PROD'. 
### Otherwise, adjustments will be necessary.

### Classical environmental index for initial impulse U_j
env_means = df.groupby('AMB')['PROD'].mean()
grand_mean = df['PROD'].mean()
U_init = env_means - grand_mean
U_values = pd.Series(U_init, name='U')
gens = df['GEN'].unique()
amb_df = df.groupby('AMB').size() 
max_iter = 2000
tol = 1e-4

def regression_step(U_values):
    new_params = {}
    new_models = {}
    for gen in gens:
        df_gen = df[df['GEN'] == gen].copy()
        df_gen = df_gen.join(U_values, on='AMB')
        df_gen['Z'] = df_gen['U'].apply(lambda x: 1 if x <= 0 else 0)
        df_gen['ZU'] = df_gen['Z'] * df_gen['U']
        df_gen['NZU'] = (1 - df_gen['Z']) * df_gen['U']
        
        X = sm.add_constant(df_gen[['ZU', 'NZU']])
        y = df_gen['PROD']
        
        model = sm.OLS(y, X).fit()
        new_params[gen] = model.params
        new_models[gen] = model
    return new_params, new_models

def update_U(params):
    new_U = {}
    for amb in amb_df.index:
        numerador = 0.0
        denominador = 0.0
        # Use U_values para determinar Z_j atual
        Zj = 1 if U_values[amb] <= 0 else 0
        df_amb = df[df['AMB'] == amb]
        for gen in gens:
            df_g = df_amb[df_amb['GEN'] == gen]
            if df_g.empty:
                continue
            a = params[gen]['const']
            b1 = params[gen]['ZU']
            b2 = params[gen]['NZU']
            b = b1 if Zj == 1 else b2
            y = df_g['PROD'].values
            numerador += np.sum((y - a) * b)
            denominador += np.sum(b**2)
        if denominador != 0:
            new_U[amb] = numerador / denominador
        else:
            new_U[amb] = U_values[amb]  
    return pd.Series(new_U, name='U')  ### ** It's important to name the series **

start_time = time.time()
for i in range(max_iter):
    new_params, new_models = regression_step(U_values)
   
    new_U = update_U(new_params)
    
    diff = np.max(np.abs(new_U - U_values))
    print(f"Iter {i+1}, max U change = {diff:.6f}")
    
    if diff < tol:
        print("Convergência atingida.")
        break
    
    U_values = new_U

end_time = time.time()
print(f"Tempo total: {end_time - start_time:.2f} s")

results = []
for gen in gens:
    df_gen = df[df['GEN'] == gen].copy()
    df_gen = df_gen.join(U_values, on='AMB')
    df_gen['Z'] = df_gen['U'].apply(lambda x: 1 if x <= 0 else 0)
    df_gen['ZU'] = df_gen['Z'] * df_gen['U']
    df_gen['NZU'] = (1 - df_gen['Z']) * df_gen['U']
    
    X = sm.add_constant(df_gen[['ZU', 'NZU']])
    y = df_gen['PROD']
    
    model = sm.OLS(y, X).fit()
    
    t_a = model.t_test('const = 0').summary_frame().iloc[0]
    t_b10 = model.t_test('ZU = 0').summary_frame().iloc[0]
    t_b11 = model.t_test('ZU = 1').summary_frame().iloc[0]
    t_b20 = model.t_test('NZU = 0').summary_frame().iloc[0]
    t_b21 = model.t_test('NZU = 1').summary_frame().iloc[0]
    
    t_b1_eq_b2 = model.t_test('ZU - NZU = 0').summary_frame().iloc[0]
    f_b1_eq_b2_res = model.f_test('ZU = NZU')
    f_stat = f_b1_eq_b2_res.fvalue if hasattr(f_b1_eq_b2_res, 'fvalue') else np.nan
    f_pval = f_b1_eq_b2_res.pvalue if hasattr(f_b1_eq_b2_res, 'pvalue') else np.nan
    
    results.append({
        'GEN': gen,
        'a_est': model.params['const'],
        'a_se': model.bse['const'],
        'b1_est': model.params['ZU'],
        'b1_se': model.bse['ZU'],
        'b2_est': model.params['NZU'],
        'b2_se': model.bse['NZU'],
        'U_mean': U_values.mean(),
        't_a_stat': t_a['t'],
        't_a_pval': t_a['P>|t|'],
        't_b1_0_stat': t_b10['t'],
        't_b1_0_pval': t_b10['P>|t|'],
        't_b1_1_stat': t_b11['t'],
        't_b1_1_pval': t_b11['P>|t|'],
        't_b2_0_stat': t_b20['t'],
        't_b2_0_pval': t_b20['P>|t|'],
        't_b2_1_stat': t_b21['t'],
        't_b2_1_pval': t_b21['P>|t|'],
        't_b1_eq_b2_stat': t_b1_eq_b2['t'],
        't_b1_eq_b2_pval': t_b1_eq_b2['P>|t|'],
        'f_b1_eq_b2_stat': f_stat,
        'f_b1_eq_b2_pval': f_pval,
    })

for res in results:
    if res['f_b1_eq_b2_pval'] > 0.05:  # Accept H0 -> use model with common beta
        gen = res['GEN']
        df_gen = df[df['GEN'] == gen].copy()
        df_gen = df_gen.join(U_values, on='AMB')
        
        # Modelo linear simples com U
        X = sm.add_constant(df_gen[['U']])
        y = df_gen['PROD']
        
        model_common = sm.OLS(y, X).fit()
        
        # Teste t para beta comum = 1
        t_beta1 = model_common.t_test('U = 1').summary_frame().iloc[0]

        res['beta_comum'] = model_common.params['U']
        res['beta_comum_se'] = model_common.bse['U']
        res['beta_comum_t'] = model_common.tvalues['U']
        res['beta_comum_pval'] = model_common.pvalues['U']
        res['t_beta_comum_eq_1_stat'] = t_beta1['t']
        res['t_beta_comum_eq_1_pval'] = t_beta1['P>|t|']
    else:
        res['beta_comum'] = np.nan
        res['beta_comum_se'] = np.nan
        res['beta_comum_t'] = np.nan
        res['beta_comum_pval'] = np.nan
        res['t_beta_comum_eq_1_stat'] = np.nan
        res['t_beta_comum_eq_1_pval'] = np.nan

param_df = pd.DataFrame(results)
#param_df.to_csv('estimated_parameters.txt', sep='\t', decimal='.', index=False)
print(param_df)

In [ ]:
### BAYESIAN INFERENCE

### Let's divide this into two steps:
### 1) considering that all genotypes fit the linear model, i.e., with a common beta)
### 2) considering that all genotypes fit the segmented model.

In [ ]:
### --- Step 1 --- ###

import numpy as np
import pandas as pd
import ultranest
from scipy.stats import norm

env_means = df.groupby('AMB')['PROD'].mean()
grand_mean = df['PROD'].mean()
U_values = env_means - grand_mean  # U(j) = média ambiente - média geral

param_df = param_df.replace([np.inf, -np.inf], np.nan)

def fit_toler_simple_bayes(df_gen, U_values, prior_means, prior_stds):
    df_gen = df_gen.copy()
    df_gen = df_gen.join(U_values.rename("U"), on='AMB')
    y = df_gen['PROD'].values
    U = df_gen['U'].values
    n = len(y)

    param_names = ['a', 'b']

    def prior_transform(cube):
        u0 = np.clip(cube[0], 1e-6, 1 - 1e-6)
        u1 = np.clip(cube[1], 1e-6, 1 - 1e-6)
        a = norm.ppf(u0, loc=prior_means['a'], scale=max(prior_stds['a'], 1e-3))
        b = norm.ppf(u1, loc=prior_means['b'], scale=max(prior_stds['b'], 1e-3))
        return [a, b]

    def loglike(params):
        a, b = params
        mu = a + b * U
        residuals = y - mu
        sigma = np.std(residuals) if n > 1 else 1.0
        if sigma <= 0 or not np.isfinite(sigma):
            return -np.inf
        return -0.5 * np.sum(np.log(2 * np.pi * sigma**2) + (residuals / sigma) ** 2)

    sampler = ultranest.ReactiveNestedSampler(param_names, loglike, prior_transform)
    result = sampler.run(min_num_live_points=400, dlogz=0.5, max_ncalls=10000)

    samples = result['samples']
    means = dict(zip(param_names, samples.mean(axis=0)))
    hdi_low = np.percentile(samples, 2.5, axis=0)
    hdi_high = np.percentile(samples, 97.5, axis=0)
    hdi = dict(zip(param_names, zip(hdi_low, hdi_high)))

    return {'means': means, 'hdi': hdi, 'samples': samples}

results_toler_simple = {}

for gen in df['GEN'].unique():
    df_gen = df[df['GEN'] == gen]
    row = param_df[param_df['GEN'] == gen].iloc[0]

    a_mean = row['a_est'] if np.isfinite(row['a_est']) else 30.0
    a_se = row['a_se'] if np.isfinite(row['a_se']) and row['a_se'] > 0 else 5.0
    b_mean = row['beta_comum'] if np.isfinite(row['beta_comum']) else 1.0
    b_se = row['beta_comum_se'] if np.isfinite(row['beta_comum_se']) and row['beta_comum_se'] > 0 else 1.0

    prior_means = {'a': a_mean, 'b': b_mean}
    prior_stds = {'a': a_se, 'b': b_se}

    print(f"Running for each genotype {gen}")
    res = fit_toler_simple_bayes(df_gen, U_values, prior_means, prior_stds)
    results_toler_simple[gen] = res

print("\nResults for beta_common (Bayesian):")
for gen, res in results_toler_simple.items():
    m = res['means']
    h = res['hdi']
    print(f"Genotype {gen}:")
    print(f" a = {m['a']:.2f}, HDI = [{h['a'][0]:.2f}, {h['a'][1]:.2f}]")
    print(f" b = {m['b']:.2f}, HDI = [{h['b'][0]:.2f}, {h['b'][1]:.2f}]\n")

In [ ]:
### --- Step 1 --- ###

import numpy as np
import pandas as pd
import pymc as pm
import arviz as az

env_means = df.groupby('AMB')['PROD'].mean()
grand_mean = df['PROD'].mean()
U_values = env_means - grand_mean  # U(j)

param_df = param_df.replace([np.inf, -np.inf], np.nan)

def fit_toler_simple_bayes_pymc(df_gen, U_values, prior_means, prior_stds):

    df_gen = df_gen.copy()
    df_gen = df_gen.join(U_values.rename("U"), on='AMB')

    y = df_gen['PROD'].values
    U = df_gen['U'].values

    with pm.Model() as model:

        # Priors informativos
        a = pm.Normal("a",
                      mu=prior_means['a'],
                      sigma=max(prior_stds['a'], 1e-3))

        b = pm.Normal("b",
                      mu=prior_means['b'],
                      sigma=max(prior_stds['b'], 1e-3))

        sigma = pm.HalfNormal("sigma", sigma=10)

        # Modelo linear
        mu = a + b * U

        # Likelihood
        y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)

        trace = pm.sample(
            draws=4000,
            tune=2000,
            target_accept=0.9,
            cores=4,
            return_inferencedata=True,
            progressbar=True
        )

    summary = az.summary(trace, hdi_prob=0.95)

    means = {
        'a': summary.loc['a', 'mean'],
        'b': summary.loc['b', 'mean']
    }

    hdi = {
        'a': (summary.loc['a', 'hdi_2.5%'], summary.loc['a', 'hdi_97.5%']),
        'b': (summary.loc['b', 'hdi_2.5%'], summary.loc['b', 'hdi_97.5%'])
    }

    samples = trace.posterior

    return {'means': means, 'hdi': hdi, 'trace': trace, 'samples': samples}


# =============================
# Loop por genótipo
# =============================

results_toler_simple = {}

for gen in df['GEN'].unique():

    df_gen = df[df['GEN'] == gen]
    row = param_df[param_df['GEN'] == gen].iloc[0]

    a_mean = row['a_est'] if np.isfinite(row['a_est']) else 30.0
    a_se = row['a_se'] if np.isfinite(row['a_se']) and row['a_se'] > 0 else 5.0

    b_mean = row['beta_comum'] if np.isfinite(row['beta_comum']) else 1.0
    b_se = row['beta_comum_se'] if np.isfinite(row['beta_comum_se']) and row['beta_comum_se'] > 0 else 1.0

    prior_means = {'a': a_mean, 'b': b_mean}
    prior_stds = {'a': a_se, 'b': b_se}

    print(f"Running for genotype {gen}")

    res = fit_toler_simple_bayes_pymc(
        df_gen,
        U_values,
        prior_means,
        prior_stds
    )

    results_toler_simple[gen] = res


# =============================
# Results
# =============================

print("\nResults for beta_common (Bayesian - PyMC):")

for gen, res in results_toler_simple.items():
    m = res['means']
    h = res['hdi']

    print(f"Genotype {gen}:")
    print(f" a = {m['a']:.2f}, HDI = [{h['a'][0]:.2f}, {h['a'][1]:.2f}]")
    print(f" b = {m['b']:.2f}, HDI = [{h['b'][0]:.2f}, {h['b'][1]:.2f}]\n")

In [ ]:
### --- Step 2 --- ###

import os
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

# ====================================================
# CHECK IF ANOVA RESULTS EXIST
# ====================================================
if 'QMR_global' not in globals():
    raise ValueError("QMR_global not found. Run ANOVA cell first.")

if 'r' not in globals():
    raise ValueError("Number of replications (r) not found. Run ANOVA cell first.")

# ====================================================
# CONFIGURATIONS
# ====================================================
output_dir = 'bayesian_adaptability_toler_step2'
os.makedirs(output_dir, exist_ok=True)

# ====================================================
# PREPROCESSING (Environmental Index μj)
# ====================================================
df_ambiental = (
    df.groupby('AMB', as_index=False)
      .agg({'PROD': 'mean'})
      .rename(columns={'PROD': 'Ij'})
)

I_mean = df_ambiental['Ij'].mean()
df_ambiental['mu_j'] = df_ambiental['Ij'] - I_mean

# Atribuição segura (sem merge)
df['mu_j'] = df['AMB'].map(
    df_ambiental.set_index('AMB')['mu_j']
)

# ====================================================
# BAYESIAN MODELING (TOLER, 1998)
# ====================================================
results = {}

for genotype in df['GEN'].unique():

    df_genotype = df[df['GEN'] == genotype]
    X = df_genotype['mu_j'].values
    y = df_genotype['PROD'].values

    with pm.Model() as model:

        alpha = pm.Normal("alpha", mu=np.mean(y), sigma=25)
        beta1 = pm.Normal("beta1", mu=1, sigma=5)
        beta2 = pm.Normal("beta2", mu=1, sigma=5)
        sigma = pm.HalfNormal("sigma", sigma=5)

        mu = alpha + beta1 * X * (X <= 0) + beta2 * X * (X > 0)

        Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=y)

        trace = pm.sample(
            draws=4000,
            tune=2000,
            target_accept=0.9,
            cores=4,
            return_inferencedata=True,
            progressbar=True
        )

    # ====================================================
    # Posterior summaries
    # ====================================================
    summary = az.summary(trace, hdi_prob=0.95)

    beta1_median = np.median(trace.posterior['beta1'].values)
    beta2_median = np.median(trace.posterior['beta2'].values)
    alpha_median = np.median(trace.posterior['alpha'].values)

    hdi_beta1 = (
        summary.loc['beta1', 'hdi_2.5%'],
        summary.loc['beta1', 'hdi_97.5%']
    )

    hdi_beta2 = (
        summary.loc['beta2', 'hdi_2.5%'],
        summary.loc['beta2', 'hdi_97.5%']
    )

    # ====================================================
    # Stability (S²d)
    # ====================================================
    alpha_samples = trace.posterior['alpha'].values.reshape(-1)
    beta1_samples = trace.posterior['beta1'].values.reshape(-1)
    beta2_samples = trace.posterior['beta2'].values.reshape(-1)

    y_preds = np.array([
        a + b1 * X * (X <= 0) + b2 * X * (X > 0)
        for a, b1, b2 in zip(alpha_samples, beta1_samples, beta2_samples)
    ])

    y_median = np.median(y_preds, axis=0)

    # n - 3 parameters (alpha, beta1, beta2)
    QMD_i = ((y - y_median) ** 2).sum() / (len(y) - 3)

    S2d = (QMD_i * r - QMR_global) / r

    # ====================================================
    # Store results
    # ====================================================
    results[genotype] = {
        'alpha_median': alpha_median,
        'beta1_median': beta1_median,
        'beta2_median': beta2_median,
        'beta1_HDI_low': hdi_beta1[0],
        'beta1_HDI_high': hdi_beta1[1],
        'beta2_HDI_low': hdi_beta2[0],
        'beta2_HDI_high': hdi_beta2[1],
        'S2d': S2d
    }

# ====================================================
# SAVE RESULTS
# ====================================================
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.to_csv(
    os.path.join(output_dir, 'bayesian_results.txt'),
    index=True,
    sep="\t",
    decimal=","
)